**Scenario**: Alice wants to send a message to Bob using RSA.

**Step 1**: Bob generates a public key $(n,e)$ where $n=pq$ is a product of large primes. An important detail is that $e$ must be invertible modulo $\phi(n) = (p-1)(q-1)$.

In [1]:
e = 65537

valid = False
while not valid:
    p = next_prime(ZZ.random_element(10^127, 10^128))
    q = next_prime(ZZ.random_element(10^127, 10^128))
    n = p * q
    phi = (p-1)*(q-1)
    if gcd(phi, e) == 1:
        valid = True

print("n = {}".format(n))
print("e = {}".format(e))

n = 1022996316199943876276108843150540399217141129248647452513958451507439380242482771904052232493285663321493655473643072726110318227954277976925513690908001465423605764113675075363834698719789104588943102843586783913592588943609849835315173351865389669923371
e = 65537


**Step 2**: Bob computes his private key $d = e^{-1} \bmod{\phi(n)}$.

In [2]:
d = e.inverse_mod(phi); d

541210740120610557063113139291019374119302336593208484879746821347726294944342320635019897233733624039593329303784924814375039301134748445909338029416271633162777128689885999857712585683767846252157061195251245224912876863215893934101695417679410355678337

**Step 3**: Alice encrypts her message for Bob using his public key. If $M$ is the message (as a number), then the ciphertext is given by $C = M^e \bmod{n}$.

In [3]:
def text_to_number(message):
    digits = [str(ord(c)).zfill(3) for c in message]
    return Integer("".join(digits), base=10)

M = "Hello, Bob. Let's meet at 7:00 tomorrow in the park. - Alice"
C = power_mod(text_to_number(M), e, n); C

730427929538661009399291638265991315191795498010416246053474735405601851802778006195431393766108516731992958020285813520248889687242995773671340994561765742061848423841515112335370090895791813959062340111844698276214378477534352533581083241774196319489160

**Step 4**: Bob now uses his private key to decrypt the ciphertext. He computes $C^d \equiv M^{ed} \equiv M \bmod{n}$. The second congruence is a consequence of Euler's Theorem together with the fact that $ed \equiv 1 \bmod{\phi(n)}$.

In [4]:
def number_to_text(num):
    num = str(num)
    # Add leading zeros to ensure proper segmentation
    while len(num) % 3 != 0:
        num = "0" + num
    message = ""
    for i in range(0, len(num), 3):
        message += chr(int(num[i:i+3]))
    return message

M_recovered = number_to_text(power_mod(C, d, n)); M_recovered

"Hello, Bob. Let's meet at 7:00 tomorrow in the park. - Alice"

Success! The secret meeting has been arranged.

But how does Bob know that the message really came from Alice, and not some malicious third party? In fact, RSA can be used not only for encryption, but also to digitally sign it.

**Scenario**: Alice wants to sign her message to Bob.

**Step 1**: Alice generates her own public/private key pair.

In [5]:
e2 = 65537

valid = False
while not valid:
    p2 = next_prime(ZZ.random_element(10^127, 10^128))
    q2 = next_prime(ZZ.random_element(10^127, 10^128))
    n2 = p2 * q2
    phi2 = (p2-1)*(q2-1)
    if gcd(phi2, e2) == 1:
        valid = True

print("n2 = {}".format(n2))
print("e2 = {}".format(e2))

d2 = e2.inverse_mod(phi2)
print("d2 = {}".format(d2))

n2 = 770927125625116750157627534072287344615863472428419164865418149391613842112244627181949829417707208968841456238798414557629280637371024665893958509598517246246523336345080806994410473803073985705284060622870745319301116411021258622650528382390507965630387
e2 = 65537
d2 = 348156463327991523176454554311877085289160492446464165624331003365176844911974979487986467205942906508549377913235526125122508159740533633356500384414171768587347521278202450631021331904812671805411007461944455425052448647756460966932205280420509996718273


**Step 2**: Alice computes the hash $H$ of the message she wishes to send.

In [6]:
from hashlib import sha256
H = int(sha256(M.encode('UTF-8')).hexdigest(), 16); H

57042791815245828132199993481351231338844362204265837177338143641469729443894

**Step 3**: Alice signs the hash using her private key, which produces a signature $S$ to send to Bob.

In [7]:
S = power_mod(H, d2, n2); S

542389115819286542768066546499426804285249931618472495445069609464871128289157849519283108883970701065725773348824767273008648196774664309118300616205391136374018136738267067607311184922206269996597935128566715778409341230470864862523699451856166111498411

**Step 4**: Bob decodes the signature using Alice's public key and verifies that the resulting hash is equal to the hash of the message he received. Only Alice could have created the valid signature, as only she knows her private key.

In [8]:
H_recovered = power_mod(S, e2, n2)
H_message = int(sha256(M_recovered.encode('UTF-8')).hexdigest(), 16)
H_recovered == H_message

True

Now Bob can rest assured that it was in fact Alice who set up the meeting.

**Remark**: Though not mentioned above, a real-life implementation of RSA **must** use message padding to be secure. The problems with the above "textbook RSA" are described in [this post](https://crypto.stackexchange.com/questions/1448/definition-of-textbook-rsa). The most blatant issue is that RSA as described above is deterministic: sending the same message twice produces the exact same ciphertext. In certain contexts, this may prove useful to an attacker.